In [1]:
import pandas as pd
import numpy as np
import re

In [8]:
df_hans = pd.read_csv('data/NFS8-Hans.tsv', sep = '\t')
df_asitane = pd.read_csv('data/NFS8-Asitane.tsv', sep = '\t')
df_galata = pd.read_csv('data/NFS8-Galata.tsv', sep = '\t')
df_yenikoy = pd.read_csv('data/NFS8-Yeniköy_İstinye_Emirgan.tsv', sep = '\t')

In [17]:
#Run this only once
df_galata.columns = df_galata.iloc[0]
df_galata = df_galata.drop(0, axis = 0)
df_galata = df_galata.loc[:, df_galata.columns.notnull()]

In [43]:
df_asitane.head(5)

,Register,Date,Unique person ID#,Entry code (page#/line/entry),Guarantor (Kefil) name(s),Guarantor (Kefil) religion (Muslim or Christian),Han Nazırı name(s),Han Name,Descriptor/location of Han,Neighborhood,...,church name,name of weapon,number of weapons,name of weapons,name of weapon.1,other info,other info.1,comments/intuitive info,Profession With ?,Profession With *
0,NFS.d_008,H 29-12-1236,NaN,4A1,NaN,NaN,NaN,Kürkçü Han,Mahmud Paşa kurbunda,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,8,H 29-12-1236,NaN,NaN,NaN,NaN,NaN,Kürkçü Han,Mahmud Paşa kurbunda,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,8,H 29-12-1236,NaN,NaN,NaN,NaN,NaN,Kürkçü Han,Mahmud Paşa kurbunda,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,8,H 29-12-1236,NaN,NaN,NaN,NaN,NaN,Kürkçü Han,Mahmud Paşa kurbunda,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,8,NaN,NaN,NaN,NaN,NaN,NaN,Kürkçü Han,Mahmud Paşa kurbunda,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [39]:
#Asitane Profession Normalization
df_asitane['Profession'] = df_asitane['Profession'].str.lower()
df_asitane['Profession With ?'] = df_asitane['Profession'].str.contains(r'\?')
df_asitane['Profession With *'] = df_asitane['Profession'].str.contains(r'\*')
df_asitane['Profession'] = df_asitane['Profession'].str.replace('[\*\(\)\?)]', '')
df_asitane['Profession'] = df_asitane['Profession'].str.replace('[\s]*', '')

/var/folders/61/v75z8wf12tscwpd8xmvjl4km0000gn/T/ipykernel_65573/1469674348.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_asitane['Profession'] = df_asitane['Profession'].str.replace('[\*\(\)\?)]', '')
/var/folders/61/v75z8wf12tscwpd8xmvjl4km0000gn/T/ipykernel_65573/1469674348.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_asitane['Profession'] = df_asitane['Profession'].str.replace('[\s]*', '')


In [38]:
#Yenikoy Profession Normalization
df_yenikoy['Profession'] = df_yenikoy['Profession'].str.lower()
df_yenikoy['Profession With ?'] = df_yenikoy['Profession'].str.contains(r'\?')
df_yenikoy['Profession With *'] = df_yenikoy['Profession'].str.contains(r'\*')
df_yenikoy['Profession'] = df_yenikoy['Profession'].str.replace('[\*\(\)\?)]', '')
df_yenikoy['Profession'] = df_yenikoy['Profession'].str.replace('[\s]*', '')

/var/folders/61/v75z8wf12tscwpd8xmvjl4km0000gn/T/ipykernel_65573/4111657177.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_yenikoy['Profession'] = df_yenikoy['Profession'].str.replace('[\*\(\)\?)]', '')
/var/folders/61/v75z8wf12tscwpd8xmvjl4km0000gn/T/ipykernel_65573/4111657177.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_yenikoy['Profession'] = df_yenikoy['Profession'].str.replace('[\s]*', '')


In [33]:
places = ['asitane', 'galata', 'yenikoy']
dfs = [df_asitane, df_galata, df_yenikoy]

In [44]:
df_asitane['Han Name']

array(['Kürkçü Han'], dtype=object)

In [55]:
#Getting rid of repeated han names
unique_hans = df_hans['Name of han'].unique()
df_hans = df_hans.drop_duplicates(subset = ['Name of han'], keep = 'first') #Before this, the hands dataframe had 176 rows but 172 unique han names
df_hans.shape

(172, 9)

In [63]:
#Split of muharrir 
split_on = r"\([\d*]\)"
muharrir_split = df_yenikoy.loc[df_yenikoy['muharrir'].str.contains(r"\([\d*]\)", na= False)]['muharrir'].str.split(split_on) #Doing split in only parenthesis formatted 
muharrir_split = muharrir_split.str[1:]
split_on_ve = r" ve "
muharrir_split_ve = df_yenikoy.loc[df_yenikoy['muharrir'].str.contains(r" ve ", na= False)]['muharrir'].str.split(split_on_ve)
muharrir_split_general = pd.concat([muharrir_split, muharrir_split_ve])
muharrir_split_general

0      [ Kürkçü Sinan Çorbacı ,  Simsar Balaş Çorbacı...
1      [ Kürkçü Sinan Çorbacı ,  Simsar Balaş Çorbacı...
2      [ Kürkçü Sinan Çorbacı ,  Simsar Balaş Çorbacı...
3      [ Kürkçü Sinan Çorbacı ,  Simsar Balaş Çorbacı...
4      [ Kürkçü Sinan Çorbacı ,  Simsar Balaş Çorbacı...
                             ...                        
705    [Panayot, Papa Iskuli, Zaferi, Yorgi, Dimitri,...
706    [Panayot, Papa Iskuli, Zaferi, Yorgi, Dimitri,...
707    [Panayot, Papa Iskuli, Zaferi, Yorgi, Dimitri,...
708    [Panayot, Papa Iskuli, Zaferi, Yorgi, Dimitri,...
709    [Panayot, Papa Iskuli, Zaferi, Yorgi, Dimitri,...
Name: muharrir, Length: 631, dtype: object

In [71]:
df_muharrir = pd.DataFrame(muharrir_split_general)
df_all_muh = pd.DataFrame(df_muharrir['muharrir'].to_list(), index = df_muharrir.index,  columns = ['Muharrir 1', 'Muharrir 2', 'Muharrir 3', 'Muharrir 4', 'Muharrir 5', 'Muharrir 6', 'Muharrir 7' , 'Muharrir 8', 'Muharrir 9', 'Muharrir 10', 'Muharrir 11'])
df_yenikoy = df_yenikoy.merge(df_all_muh, left_index = True, right_index = True)
df_yenikoy = df_yenikoy.drop('muharrir', axis = 1)

In [73]:
df_yenikoy = df_yenikoy.fillna('uknown')

In [76]:
df_galata = df_galata.fillna('uknown')